<center><img src="Fig/Ensimag.png" width="30%" height="30%"></center>
<center><h3>ENSIMAG  -- 2A</h3></center>
<hr>
<center><h1>Optimisation Numérique</h1></center>
<center><h2>TP : Linear and Quadratic programs (1.5h)</h2></center>

# Structure of an optimization program

An optimization program can be practically divided into three parts:
* the *run* environment, in which you test, run your program, and display results.
* the *problem* part, which contains the function oracles, problem constraints, etc.
* the *algorithmic* part, where the algorithms are coded.

The main interest of such division is that these parts are interchangeable, meaning that, for instance, the algorithms of the third part can be used of a variety of problems. That is why such a decomposition is widely used.

In the present lab, you will use this division:
* `LP_and_QP_problems.ipynb` will be the *run* environment
* `toy_problem.ipynb` will be the considered the *problem* for this lab
* the library `cvxopt` will be used for solving all optimization problems

---

The following script will allow you to import *notebooks* as if you imported *python files* and will have to be executed at each time you launch Jupyter notebooks.

In [ ]:
import start
import import_ipynb
from importlib import reload

---

# Regression model


We consider the regression model

$$ y=X\theta+\xi,\;\;\xi\sim \mathcal{N}(0, \sigma I_m), $$

where $X\in \mathbb{R}^{m\times n}$ and $y\in \mathbb{R}^m$ are the observed values and $\theta\in \mathbb{R}^n$ is the unknown parameter we want to find. 

We want to find a value $\widehat{\theta}$ such that 
- the quadratic error $e(\widehat{\theta}) = 1/2 \|X\widehat{\theta} - y \|_2^2$ is (near)-minimal, that is $ \| \nabla e(\widehat{\theta}) \| = \| X^\mathrm{T} (X\widehat{\theta} - y) \| $ is small.

---

# the Dantzig Selector


The **Dantzig Selector** for $\theta$, introduced in *Emmanuel Candes and Terrence Tao "The Dantzig selector: Statistical estimation when $p$ is much larger than $n$". The Annals of Statistics, 2007* can be used to estimate $\theta$ in the case of an overparametrized problem, i.e. when the dimension $n$ of $\theta$ is well greater than the dimension of the observation $y$. 




In that case, the estimator $\widehat{\theta}_{DS}$ is the solution of the optimization problem 
$$
\widehat{\theta}_{DS} \in \arg\min_{\theta\in \mathbb{R}^n} \left\{\|\theta\|_1,\;\mbox{with}\;\|X^T(X\theta-y)\|_\infty\leq \kappa\sigma\right\},
$$
where $\kappa>0$ is an *hyper-parameter*. 


The best theoretical value for $\kappa$ is $\nu q_{\mathcal{N}} \left(1-\frac{\alpha}{2n}\right)$, where  $\alpha\in(0,1)$ is the chosen risk level (e.g. $\alpha=.05$), $q_\mathcal{N}(p)$ is the $p$-quantile of the standard normal distribution, and $\nu=\max_j\|[X]_j\|_2$ is the maximal column norm of matrix $X$.


**Objective:** Implement a function that return the Dantzig estimator $\widehat{\theta}_{DS}$ from input $(y,X,\sigma)$ using linear programming solver `cvxopt`.

> Reformulate the $\arg\min$ problem as a linear program.


> Write a function `DantzigSelector` that takes `y,X,sigma` as an input and outputs $\widehat{\theta}_{DS}$ by using the linear program solver of the library `cvxopt` <a href="http://cvxopt.org/userguide/coneprog.html#linear-programming">lp</a> which, given $c, G, h$ solves the problem
$$ \min_x c^\mathrm{T}x ~~~~~~ \text{ subject to } Gx \leq h $$
where the inequality is elementwise.

*Hint: Useful functions are* `np.concatenate` `np.zeros` `np.eye` `np.hstack` `np.vstack`

In [6]:
from cvxopt import matrix, solvers

from scipy.stats import norm
import numpy as np

def DantzigSelector(y,X,sigma):
    
    # Extracting the sizes
    m,n = X.shape
    
    # Computing kappa
    alpha = 0.05
    nu = max(np.linalg.norm(X, axis=0))
    kappa = nu*norm.ppf(1-alpha/(2.0*n))
    
    
    #####################################################
    # COMPUTE AND SOLVE LP PROBLEM
    #####################################################
    # Prepare c, G, h
    c = np.ones(3*n)
    assert c.shape == (3*n,)
    G = np.concatenate([X.T*X, -X.T*X], [-X.T*X, X.T*X])
    assert G.shape == (7*n, 3*n)
    h = np.concatenate(kappa*sigma*np.ones() + X.T*y , kappa*sigma*np.ones() - X.T*y)
    assert h.shape == (7*n,)
    
    c_cvx = matrix(c)
    G_cvx = matrix(G)
    h_cvx = matrix(h)
    
    sol = solvers.conelp(c_cvx, G_cvx, h_cvx)
    x_cvx = sol['x']
    x = np.array(x_cvx)
    assert x.shape == (3*n,)
    
    theta = np.concatenate([]) # x[n:2*n]
    assert theta.shape == (n,)
    
    return theta

> Test your code on the toy problem given in `toy_problem.ipynb`. For this randomly generated problem, there is a true $\theta$ upon which the problem is built.

In [7]:
import toy_problem as tPb
reload(tPb)

theta_ds = DantzigSelector(tPb.y,tPb.X,tPb.sigma)


ValueError: operands could not be broadcast together with shapes (256,72) (72,256) 

> Investigate the differences between the Dantzig selector and the Ground truth

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

plt.figure()
plt.plot(tPb.theta,'ro',label='ground truth')
plt.plot(theta_ds,'b*',label='Dantzig selector')
plt.legend()
plt.show()

---

# the Lasso

Under the same regression model, the Least Absolute Shrinkage and Selection Operator or **lasso** for $\theta$, introduced in *Robert Tibshirani "Regression shrinkage and selection via the lasso", Journal of the Royal Statistical Society, 1996* can also be used to estimate $\theta$. 

The estimator $\widehat{\theta}_{L}$ is the solution of the optimization problem 
$$
\widehat{\theta}_{L} \in \arg\min_{\theta\in \mathbb{R}^n} \left\{ \|X\theta - y\|_2^2 + \kappa \sigma \|\theta\|_1 \right\},
$$
where $\kappa>0$ is an *hyper-parameter*. 


The best theoretical value for $\kappa$ is the same as for the Dantzig selector: $\nu q_{\mathcal{N}} \left(1-\frac{\alpha}{2n}\right)$, where  $\alpha\in(0,1)$ is the chosen risk level (e.g. $\alpha=.05$), $q_\mathcal{N}(p)$ is the $p$-quantile of the standard normal distribution, and $\nu=\max_j\|[X]_j\|_2$ is the maximal column norm of matrix $X$.


**Objective:** Implement a function that return the lasso estimator $\widehat{\theta}_{L}$ from input $(y,X,\sigma)$ using quadratic programming solver `cvxopt`.


> Reformulate the $\arg\min$ problem as a quadratic program.


> Write a function `Lasso` that takes `y,X,sigma` as an input and outputs $\widehat{\theta}_{L}$ by using the quadratic program solver of the library `cvxopt` <a href="http://cvxopt.org/userguide/coneprog.html#quadratic-programming">qp</a> which, given $P,q, G, h$ solves the problem
$$ \min_x 1/2 x^\mathrm{T} P x + q^\mathrm{T}x ~~~~~~ \text{ subject to } Gx \leq h $$
where the inequality is elementwise.

*Hint: Useful functions are* `np.concatenate` `np.zeros` `np.eye` `np.hstack` `np.vstack`

In [ ]:
from cvxopt import matrix, solvers

from scipy.stats import norm
import numpy as np

def Lasso(y,X,sigma):
    
    # Extracting the sizes
    m,n = X.shape
    
    # Computing kappa
    alpha = 0.05
    nu = max(np.linalg.norm(X, axis=0))
    kappa = nu*norm.ppf(1-alpha/(2.0*n))
    
    #####################################################
    # COMPUTE AND SOLVE QP PROBLEM
    #####################################################
    # Prepare P, q, G, h
    # assert P.shape == (2*n, 2*n)
    # assert q.shape == (2*n,)
    # assert G.shape == (3*n, 2*n)
    # assert h.shape == (3*n,)
    P_cvx = matrix(P)
    q_cvx = matrix(q)
    G_cvx = matrix(G)
    h_cvx = matrix(h)
    
    sol = solvers.qp(P_cvx, q_cvx, G_cvx, h_cvx)
    x_cvx = sol['x']
    # x = np.squeeze(np.array(x_cvx))
    assert x.shape == (2*n,)
    
    theta = ... # x[:n]
    assert theta.shape == (n,)
    
    return theta

> Test your code on the toy problem given in `toy_problem.ipynb`. For this randomly generated problem, there is a true $\theta$ upon which the problem is built.

In [3]:
import toy_problem as tPb
reload(tPb)


theta_l = Lasso(tPb.y,tPb.X,tPb.sigma)


NameError: name 'reload' is not defined

> Investigate the differences between the Lasso and the Ground truth

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

plt.figure()
plt.plot(tPb.theta,'ro',label='ground truth')
plt.plot(theta_l,'k*',label='Lasso')
plt.legend()
plt.show()

# Comparing and Improving


> Compute the lasso and Dantzig selector solutions for the same problem and investigate their compared performance graphically and by computing the error on the null and non-null coordinates of theta.


> Play with the values of $n,m,\sigma, S$ to exhibit differences in behaviors.


In [ ]:
import toy_problem as tPb
reload(tPb)


theta_l = Lasso(tPb.y,tPb.X,tPb.sigma)
theta_ds = DantzigSelector(tPb.y,tPb.X,tPb.sigma)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

plt.figure()
plt.plot(tPb.theta,'ro',label='ground truth')
plt.plot(theta_ds,'b*',label='Dantzig selector')
plt.plot(theta_l,'k*',label='Lasso')
plt.legend()
plt.show()



In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline



plt.figure()
plt.plot(tPb.theta[tPb.non_null],'ro',label='ground truth')
plt.plot(theta_ds[tPb.non_null],'b*',label='Dantzig selector')
plt.plot(theta_l[tPb.non_null],'k*',label='Lasso')
plt.legend()
plt.show()

err_theta_DS = np.linalg.norm(theta_ds[tPb.non_null] -tPb.theta[tPb.non_null]  , ord=1)
err_theta_L = np.linalg.norm(theta_l[tPb.non_null]-tPb.theta[tPb.non_null] , ord=1)

print("Error on the non-null coefficients\n Dantzig selec. \t Lasso")
print(err_theta_DS,err_theta_L)

In [ ]:
err_supp_DS = 0
err_supp_L = 0
for i in range(tPb.n):
    if i not in  tPb.non_null:
        err_supp_DS += np.abs(theta_ds[i])
        err_supp_L += np.abs(theta_l[i])

print("Error on the null coefficients\n Dantzig selec. \t Lasso")
print(float(err_supp_DS),float(err_supp_L))
        

In [ ]:
print("Error on y \n Dantzig selec. \t Lasso")
print(float(np.linalg.norm(np.dot(tPb.X, theta_ds ) - tPb.y )),float(np.linalg.norm(np.dot(tPb.X, theta_l ) - tPb.y )))   

In [ ]:
print("Quadratic error \n Dantzig selec. \t Lasso")
print(float(np.linalg.norm(np.dot(tPb.X.T , np.dot(tPb.X, theta_ds ) - tPb.y ))),float(np.linalg.norm(np.dot(tPb.X.T , np.dot(tPb.X, theta_l ) - tPb.y )))) 

# To go further


### Improved Estimators

A popular improvement of these estimators is to weigh the threshold $\kappa$ with the norm corresponding column of $X$.

The improved Dantzig and Lasso estimators $\widehat{\theta'}_{DS}$ and $\widehat{\theta'}_{L}$  are thus solutions of  
$$
\widehat{\theta}_{DS} \in \arg\min_{\theta\in \mathbb{R}^n} \left\{\|\theta\|_1,\;{with}\;| [ X^T(X\theta-y)]_i  | \leq \kappa_i \sigma ~~ \forall i\right\},
$$
and
$$
\widehat{\theta}_{L} \in \arg\min_{\theta\in \mathbb{R}^n} \left\{ \|X\theta - y\|_2^2 + \sum_{i=1}^n \kappa_i \sigma |\theta_i | \right\},
$$
where $\kappa_i = \| X_i \|_2 \kappa = \| X_i \|_2 q_{\mathcal{N}} \left(1-\frac{\alpha}{2n}\right) $ where $\| X_i \|_2$ is the $2$-norm of the $i$-th column of $X$.



> Investigate the pertinence of this improvement. (Get rid of the normalization in the problem). 

In [ ]:
from cvxopt import matrix, solvers

from scipy.stats import norm
import numpy as np

def DantzigSelectorImproved(y,X,sigma):
    
    # Extracting the sizes
    m,n = X.shape
    
    # Computing kappa
    alpha = 0.05
    #### WARNING: Error in the question !
    nu = 1 #max(np.linalg.norm(X, axis=0))
    kappa = nu*norm.ppf(1-alpha/(2.0*n))
    
    # Kappa vector
    kappa_vec = np.linalg.norm(X, ord=2, axis=0) * kappa
    assert kappa_vec.shape == (n,)
    
    #####################################################
    # COMPUTE AND SOLVE LP PROBLEM
    #####################################################
    # Prepare c, G, h
    c_cvx = matrix(c)
    G_cvx = matrix(G)
    h_cvx = matrix(h)
    
    sol = solvers.conelp(c_cvx, G_cvx, h_cvx)
    x_cvx = sol['x']
    x = np.squeeze(np.array(x_cvx))
    assert x.shape == (2*n,)
    
    theta = ... # x[:n]
    assert theta.shape == (n,)
    
    return  theta

In [ ]:
from cvxopt import matrix, solvers

from scipy.stats import norm
import numpy as np

def LassoImproved(y,X,sigma):
    
    # Extracting the sizes
    m,n = X.shape
    
    # Computing kappa
    alpha = 0.05
    nu = 1 # max(np.linalg.norm(X, axis=0))
    kappa = nu*norm.ppf(1-alpha/(2.0*n))
    
    # Kappa vector
    kappa_vec = np.linalg.norm(X, ord=2, axis=0) * kappa
    assert kappa_vec.shape == (n,)
    
    #####################################################
    # COMPUTE AND SOLVE QP PROBLEM
    #####################################################
    # Prepare P, q, G, h
    P_cvx = matrix(P)
    q_cvx = matrix(q)
    G_cvx = matrix(G)
    h_cvx = matrix(h)
    
    sol = solvers.qp(P_cvx, q_cvx, G_cvx, h_cvx)
    x_cvx = sol['x']
    x = np.squeeze(np.array(x_cvx))
    assert x.shape == (2*n,)
    
    theta = x[:n]
    assert theta.shape == (n,)
    
    return theta

In [ ]:
import toy_problem as tPb
reload(tPb)

theta_l = Lasso(tPb.y,tPb.X,tPb.sigma)
theta_ds = DantzigSelector(tPb.y,tPb.X,tPb.sigma)
theta_li = LassoImproved(tPb.y,tPb.X,tPb.sigma)
theta_dsi = DantzigSelectorImproved(tPb.y,tPb.X,tPb.sigma)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

plt.figure()
plt.plot(tPb.theta,'ro',label='ground truth')
plt.plot(theta_ds,'b*',label='Dantzig selector')
plt.plot(theta_l,'k*',label='Lasso')
plt.plot(theta_dsi,'g*',label='Improved Dantzig selector')
plt.plot(theta_li,'y*',label='Improved Lasso')
plt.legend()
plt.show()

In [ ]:
err_theta_DS = np.linalg.norm(theta_ds[tPb.non_null] -tPb.theta[tPb.non_null]  , ord=1)
err_theta_L = np.linalg.norm(theta_l[tPb.non_null]-tPb.theta[tPb.non_null] , ord=1)
err_theta_DSI = np.linalg.norm(theta_dsi[tPb.non_null] -tPb.theta[tPb.non_null]  , ord=1)
err_theta_LI = np.linalg.norm(theta_li[tPb.non_null]-tPb.theta[tPb.non_null] , ord=1)

print("Error on the non-null coefficients")
print(err_theta_DS,err_theta_L, err_theta_DSI, err_theta_LI)

### Crossed Validation of the thresholds

The values of $\kappa$ is heavily linked to the noise standard deviation $\sigma$ and theoretical considerations about $X$ and $\theta$. A practical way to choose $\kappa$ without the knowledge of $\sigma$ is to use *cross-validation*.

- Split y and X in two parts column-wise, one part for *training* and one for *testing*.
- For several values of $\kappa$, solve the estimation problem *on the training part* using $\kappa$ to get a estimate $\widehat{\theta}_\kappa$.
- For each $\widehat{\theta}_\kappa$, compute the error *on the testing part* $e(\kappa) = \| y_{test}-X_{test}\widehat{\theta}_\kappa \|$.  
- Choose the value that minimize this error $\widehat{\kappa} = \arg \min_{\kappa} e(\kappa)$.


> Implement a cross validation procedure to choose $\kappa$. Compare the value and the outputted estimate with the theoretic value.